In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model
from sklearn.metrics import roc_auc_score

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined_test\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined_test/data preparation/KY_cleaned").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined_test/data preparation/FL_cleaned").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['property_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['property_two_year'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.7]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [5]:
np.mean(cart_summary['FL_score']), np.mean(ebm_summary['FL_score'])

(0.6155628362643539, 0.6698951256725287)

### Lasso Stumps

In [4]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined_test/data preparation/KY_stumps").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined_test/data preparation/FL_stumps").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['property_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['property_two_year'].values

## columns 
cols = KY_X.columns[1:]

In [5]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.004, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge18',
 'age_at_current_charge19',
 'age_at_current_charge20',
 'age_at_current_charge21',
 'age_at_current_charge24',
 'age_at_current_charge27',
 'age_at_current_charge30',
 'age_at_current_charge36',
 'age_at_current_charge40',
 'age_at_current_charge43',
 'age_at_current_charge47',
 'age_at_current_charge51',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest6',
 'p_arrest7',
 'p_arrest8',
 'p_charges1',
 'p_charges5',
 'p_charges6',
 'p_violence1',
 'p_felony1',
 'p_felony2',
 'p_felony3',
 'p_misdemeanor1',
 'p_property1',
 'p_property2',
 'p_drug1',
 'p_stealing1',
 'p_fta_two_year1',
 'p_pending_charge1',
 'p_probation1',
 'p_probation2',
 'p_incarceration1',
 'five_year1']

In [6]:
c_grid={'C': [0.001, 0.002, 0.003, 0.004]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [7]:
stumps_summary['best_params'], np.mean(stumps_summary['FL_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.004}, {'C': 0.004}, {'C': 0.004}, {'C': 0.004}, {'C': 0.004}],
 0.6656102144061672,
 0.0009765850681780375)

### RiskSLIM

In [8]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.0005, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

14

In [9]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [10]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'property_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/29/19 @ 12:17 PM | 193 rows in lookup table
11/29/19 @ 12:17 PM | ------------------------------------------------------------
11/29/19 @ 12:17 PM | runnning initialization procedure
11/29/19 @ 12:17 PM | ------------------------------------------------------------
11/29/19 @ 12:17 PM | CPA produced 2 cuts
11/29/19 @ 12:17 PM | running naive rounding on 6 solutions
11/29/19 @ 12:17 PM | best objective value: 0.5590
11/29/19 @ 12:17 PM | rounding produced 5 integer solutions
11/29/19 @ 12:17 PM | best objective value is 0.5731
11/29/19 @ 12:17 PM | running sequential rounding on 6 solutions
11/29/19 @ 12:17 PM | best objective value: 0.5590
11/29/19 @ 12:17 PM | sequential rounding produced 3 integer solutions
11/29/19 @ 12:17 PM | best objective value: 0.3637
11/29/19 @ 12:17 PM | polishing 8 solutions
11/29/19 @ 12:17 PM | best objective value: 0.3637
11/29/19 @ 12:17 PM | polishing produced 5 integer solutions
11/29/19 @ 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/29/19 @ 12:17 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2112.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

11/29/19 @ 12:18 PM | ------------------------------------------------------------
11/29/19 @ 12:18 PM | 193 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.20672423612617497
CPXPARAM_MIP_Tolerances_UpperCutoff              0.21503233438431543


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/29/19 @ 12:18 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2150.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Sequential b&c:
  Real time             =   36.31 sec. (39680.94 ticks)
                          ------------
Total (root+branch&cut) =   36.39 sec. (39682.27 ticks)
setting c0 = 0.0 to ensure that intercept is not penalized
11/29/19 @ 12:18 PM | 193 rows in lookup table
11/29/19 @ 12:18 PM | ------------------------------------------------------------
11/29/19 @ 12:18 PM | runnning initialization procedure
11/29/19 @ 12:18 PM | ------------------------------------------------------------
11/29/19 @ 12:18 PM | CPA produced 2 cuts
11/29/19 @ 12:18 PM | running naive rounding on 6 solutions
11/29/19 @ 12:18 PM | best objective value: 0.5629
11/29/19 @ 12:18 PM | rounding produced 5 integer solutions
11/29/19 @ 12:18 PM | best objective value is 0.5730
11/29/19 @ 12:18 PM | running sequential rounding on 6 solutions
11/29/19 @ 12:18 PM | best objective value: 0.5629
11/29/19 @ 12:18 PM | sequential rounding produced 3 integer solutions
11/29/19 @ 12:18 PM | best objective value: 0.3636
1

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/29/19 @ 12:32 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2127.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.20595635839426701
CPXPARAM_MIP_Tolerances_UpperCutoff              0.21403626011586852


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/29/19 @ 12:33 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2140.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/29/19 @ 12:33 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2151.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/29/19 @ 12:34 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2135.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [11]:
np.mean(riskslim_summary['KY_validation']), np.mean(riskslim_summary['FL_score'])

(0.7466770656389105, 0.6146335822693327)

In [15]:
#### save results
summary_property2_ky_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary}

In [16]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Two Year\\"
results = [["Property",             
            np.str(round(np.mean(cart_summary['FL_score']),3)) + " (" + np.str(round(np.std(cart_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['FL_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['FL_score']),3)) + " (" + np.str(round(np.std(stumps_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['FL_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['FL_score']),3)) + ")"]]
with open(path + 'Two Year KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)